# Line Search SQP Algorithm

Implemented from Algorithm 18.3 from Nocedal (2008), p. 545.

## Problem formulation

We attempt to solve the general nonlinear programming problem (`18.10`)
 
$$ \min f(x) $$

subject to
$$ 
\begin{align*}
    c_i(x) &= 0, \quad i \in \mathcal{E}, \\
    c_i(x) &\geq 0, \quad i \in \mathcal{I}. \\
\end{align*}
$$

To model this problem we now linearize both the inequality and equality
constraints to obtain (`18.11`)

$$ 
    \min_p f_k + \nabla f_k^T p
    + \frac{1}{2} p^T \nabla_{xx}^2 \mathcal{L}_k p
$$

subject to
$$ 
\begin{align*}
    \nabla c_i(x_k)^T p + c_i(x_k) &= 0, \quad i \in \mathcal{E}, \\
    \nabla c_i(x_k)^T p + c_i(x_k) &\geq 0, \quad i \in \mathcal{I}. \\
\end{align*}
$$

We can use one of the algorithms for quadratic programming to solve this
problem. The new iterate is given by $(x_k + p_k, \lambda_{k+1})$ where $p_k$
and $\lambda_{k+1}$ are the solution and the corresponding Lagrange multiplier 
of (`18.11`)

In [ ]:
import numpy as np
from optimus import ls_sqp
from scipy.optimize import rosen, rosen_der, minimize, LinearConstraint

Let's try an example with the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function):

$$ f(x_1, x_2) = (a - x_1)^2 + b(x_2 - x_1^2)^2 $$

with global minumum at $(a,a^2)$.

In [ ]:
A = np.array([
    [ 1, -2],
    [-1, -1],
    [-1,  2],
    [ 1,  0],
    [ 0,  1]
])
b = np.array([-2,-6,-2,0,0])

In [ ]:
def fun(x):
    return rosen(x), rosen_der(x)

def restr(x):
    return np.dot(A, x) - b, A

x0 = np.array([2.,0.])

## Example using SciPy

In [ ]:
constr = LinearConstraint(A, b, [np.inf]*5)
res = minimize(fun=fun, x0=x0, method='trust-constr', jac=True, constraints=constr)
res.x # should be (1., 1.)

In [ ]:
np.dot(A, res.x) >= b

In [ ]:
x, lam = ls_sqp(fun, restr, x_0=x0, lam_0=np.ones(5), B_0=np.eye(x0.size), eta=0.4, tau=0.7, maxiters=1000, tol=10e-10)

In [ ]:
x

In [ ]:
np.dot(A, x) >= b

## A more complicated example

In [ ]:
n = 50

In [ ]:
def fun(x):
    return rosen(x), rosen_der(x)

def restr2(x: np.ndarray) -> tuple[float, np.ndarray]:
        '''
        Evaluate the restrictions of the problem: c(x) >= 0.

        Since we want that 0 <= x_i <= 1 for every i, then
            c(x) = [x_1, x_2, ..., x_n, 1 - x_1, 1 - x_2, ..., 1 - x_n]
        '''
        c = np.concatenate([x, 1 - x])
        A = np.concatenate([np.eye(x.size), -np.eye(x.size)])
        return c, A

### With SciPy

In [ ]:
n = 10

In [ ]:
x0 = np.array([2.]*n)

In [ ]:
bounds = [(0.,1.)]*n
res = minimize(fun=fun, x0=x0, method='trust-constr', jac=True, bounds=bounds)
res.x # should be (1,1,...,1)

In [ ]:
res.niter, res.fun

In [ ]:
x, lam = ls_sqp(fun, restr2, x_0=x0, lam_0=np.ones(2 * x0.size), B_0=np.eye(x0.size), eta=0.4, tau=0.7, maxiters=1000, tol=10e-4)

In [ ]:
x